Please go through the "building_strategies" notebook before looking at this notebook.

Lets work on optimizing our strategy parameters for the Bollinger Band&copy; we previously built.  We will include a function from the examples folder called build_example_strategy which builds this strategy.

In [1]:
import math
from types import SimpleNamespace
import pandas as pd
import numpy as np
import pyqstrat as pq
from pyqstrat.evaluator import compute_sharpe, compute_sortino, compute_maxdd_pct, compute_amean, compute_rolling_dd
from pyqstrat.evaluator import compute_periods_per_year
from build_example_strategy import build_example_strategy

pq.ContractGroup.clear()

Lets try to optimize the the lookback period for the moving average and the number of standard deviations for the bands.

We will try to optimize the sharpe ratio but also look at the sortino and max drawdown as we optimize the sharpe.

To do this, we have to write a generator function and a cost function.  The generator produces all the combinations of parameters you want to optimize.  The cost function will run the strategy for each parameter combination provided by the generator and return whatever metric you want to optimize, as well as any other metrics you want to see at the same time.

In this case, our cost metric will be the sharpe ratio of the strategy but we will also look at sortino and drawdowns at the same time. We also look at the number of trades generate and ignore results as unreliable if the number of trades is less than 10.

The optimizer uses multiple processes to run as fast as possible.  You can set the number of processes you want to use using the max_processes argument to the Optimizer constructor.  If you don't set this, the optimizer will the same number of processes as the CPU cores on your machine.  You may want to increase this number if your backtesting is I/O bound, and the CPU cores are often idle waiting for disk or other resources.

In this case, we are optimizing 2 parameters at the same time, but we can optimize 1 parameter or more than 2 as well.

In [9]:
def generator():
    for stop_pct in np.arange(0.001, 0.005, 0.001):
        for ret_threshold in np.arange(0, 0.006, 0.001): # number of standard deviations the bands are away from the SMA
            costs = (yield {'stop_pct' : stop_pct, 'ret_threshold' : ret_threshold})
            yield
            
def cost_func(suggestion):
    strategy = build_example_strategy(stop_pct=suggestion['stop_pct'], ret_threshold=suggestion['ret_threshold'])
    strategy.run()
    
    returns_df = strategy.df_returns().set_index('timestamp')
    
    num_trades = len(strategy.df_trades())
    
    if num_trades < 10: return np.nan, {}
    
    returns = returns_df.ret.values
    equity = returns_df.equity.values
    dates = returns_df.index.values
    
    periods_per_year = compute_periods_per_year(dates)
    
    amean = compute_amean(returns, periods_per_year)
    sharpe = compute_sharpe(returns, amean, periods_per_year)
    sortino = compute_sortino(returns, amean, periods_per_year)
    rolling_dd = compute_rolling_dd(dates, equity)
    maxdd = compute_maxdd_pct(rolling_dd[1])
    
    return sharpe, {'sortino' : sortino, 'maxdd' : maxdd, 'num_trades' : num_trades}

optimizer = pq.Optimizer('example', generator(), cost_func, max_processes=8)
optimizer.run(raise_on_error = True)

[2023-10-18 15:32:29.829 __call__] Trade: 781 2023-01-05T09:32 AAPL 2023-01-05 09:32:00 qty: 790170 prc: 126.505 order: AAPL 2023-01-05 09:31:00 qty: 790170 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:29.829 __call__] Trade: 781 2023-01-05T09:32 AAPL 2023-01-05 09:32:00 qty: 790170 prc: 126.505 order: AAPL 2023-01-05 09:31:00 qty: 790170 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:29.830 __call__] Trade: 781 2023-01-05T09:32 AAPL 2023-01-05 09:32:00 qty: 790170 prc: 126.505 order: AAPL 2023-01-05 09:31:00 qty: 790170 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:29.831 __call__] Trade: 781 2023-01-05T09:32 AAPL 2023-01-05 09:32:00 qty: 790170 prc: 126.505 order: AAPL 2023-01-05 09:31:00 qty: 790170 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:29.832 __call__] Trade: 783 2023-01-05T09:34 AAPL 2023-01-05 09:34:00 qty: -790170 prc: 125.67 order: AAPL 2023-01-05 09:33:00 qty: -790170 STOPPED_OUT OrderStatus.OPEN
[2023-10-18 15:32:29.831 __call__

[2023-10-18 15:32:29.850 __call__] Trade: 2731 2023-01-12T09:32 AAPL 2023-01-12 09:32:00 qty: 136968 prc: 132.945 order: AAPL 2023-01-12 09:31:00 qty: 136968 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:29.849 __call__] Trade: 4305 2023-01-19T09:46 AAPL 2023-01-19 09:46:00 qty: -387917 prc: 134.31 order: AAPL 2023-01-19 09:45:00 qty: -387917 STOPPED_OUT OrderStatus.OPEN
[2023-10-18 15:32:29.850 __call__] Trade: 4681 2023-01-20T09:32 AAPL 2023-01-20 09:32:00 qty: 583890 prc: 134.575 order: AAPL 2023-01-20 09:31:00 qty: 583890 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:29.850 __call__] Trade: 4305 2023-01-19T09:46 AAPL 2023-01-19 09:46:00 qty: -800551 prc: 134.31 order: AAPL 2023-01-19 09:45:00 qty: -800551 STOPPED_OUT OrderStatus.OPEN
[2023-10-18 15:32:29.852 __call__] Trade: 3511 2023-01-17T09:32 AAPL 2023-01-17 09:32:00 qty: 271485 prc: 134.99 order: AAPL 2023-01-17 09:31:00 qty: 271485 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:29.852 __call__] Tr

[2023-10-18 15:32:29.869 __call__] Trade: 4682 2023-01-20T09:33 AAPL 2023-01-20 09:33:00 qty: -129580 prc: 134.46 order: AAPL 2023-01-20 09:32:00 qty: -129580 STOPPED_OUT OrderStatus.OPEN
[2023-10-18 15:32:29.869 __call__] Trade: 3518 2023-01-17T09:39 AAPL 2023-01-17 09:39:00 qty: -145873 prc: 134.62 order: AAPL 2023-01-17 09:38:00 qty: -145873 STOPPED_OUT OrderStatus.OPEN
[2023-10-18 15:32:29.870 __call__] Trade: 5851 2023-01-25T09:32 AAPL 2023-01-25 09:32:00 qty: 961125 prc: 139.56 order: AAPL 2023-01-25 09:31:00 qty: 961125 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:29.870 __call__] Trade: 4291 2023-01-19T09:32 AAPL 2023-01-19 09:32:00 qty: 106202 prc: 134.77 order: AAPL 2023-01-19 09:31:00 qty: 106202 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:29.871 __call__] Trade: 5914 2023-01-25T10:35 AAPL 2023-01-25 10:35:00 qty: -455226 prc: 139.285 order: AAPL 2023-01-25 10:34:00 qty: -455226 STOPPED_OUT OrderStatus.OPEN
[2023-10-18 15:32:29.872 __call__] Trade: 663

[2023-10-18 15:32:29.893 __call__] Trade: 6631 2023-01-27T09:32 AAPL 2023-01-27 09:32:00 qty: 265038 prc: 144.24 order: AAPL 2023-01-27 09:31:00 qty: 265038 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:29.894 __call__] Trade: 5459 2023-01-23T16:00 AAPL 2023-01-23 16:00:00 qty: -62192 prc: 141.05 order: AAPL 2023-01-23 15:59:00 qty: -62192 EOD OrderStatus.OPEN
[2023-10-18 15:32:29.897 __call__] Trade: 5851 2023-01-25T09:32 AAPL 2023-01-25 09:32:00 qty: 342271 prc: 139.56 order: AAPL 2023-01-25 09:31:00 qty: 342271 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:29.896 __call__] Trade: 5849 2023-01-24T16:00 AAPL 2023-01-24 16:00:00 qty: -156504 prc: 142.66 order: AAPL 2023-01-24 15:59:00 qty: -156504 EOD OrderStatus.OPEN
[2023-10-18 15:32:29.897 __call__] Trade: 5461 2023-01-24T09:32 AAPL 2023-01-24 09:32:00 qty: 156504 prc: 141.151 order: AAPL 2023-01-24 09:31:00 qty: 156504 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:29.897 __call__] Trade: 7019 2023-01-2

[2023-10-18 15:32:29.965 __call__] Trade: 3121 2023-01-13T09:32 AAPL 2023-01-13 09:32:00 qty: 188287 prc: 131.77 order: AAPL 2023-01-13 09:31:00 qty: 188287 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:29.966 __call__] Trade: 3121 2023-01-13T09:32 AAPL 2023-01-13 09:32:00 qty: 183981 prc: 131.77 order: AAPL 2023-01-13 09:31:00 qty: 183981 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:29.966 __call__] Trade: 1951 2023-01-10T09:32 AAPL 2023-01-10 09:32:00 qty: 256843 prc: 130.74 order: AAPL 2023-01-10 09:31:00 qty: 256843 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:29.967 __call__] Trade: 3122 2023-01-13T09:33 AAPL 2023-01-13 09:33:00 qty: -188287 prc: 132.031 order: AAPL 2023-01-13 09:32:00 qty: -188287 STOPPED_OUT OrderStatus.OPEN
[2023-10-18 15:32:29.967 __call__] Trade: 2731 2023-01-12T09:32 AAPL 2023-01-12 09:32:00 qty: 249762 prc: 132.945 order: AAPL 2023-01-12 09:31:00 qty: 249762 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:29.968 __cal

...

[2023-10-18 15:32:30.016 __call__] Trade: 5914 2023-01-25T10:35 AAPL 2023-01-25 10:35:00 qty: -414169 prc: 139.285 order: AAPL 2023-01-25 10:34:00 qty: -414169 STOPPED_OUT OrderStatus.OPEN
[2023-10-18 15:32:30.016 __call__] Trade: 7021 2023-01-30T09:32 AAPL 2023-01-30 09:32:00 qty: 492461 prc: 145.345 order: AAPL 2023-01-30 09:31:00 qty: 492461 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:30.018 __call__] Trade: 4291 2023-01-19T09:32 AAPL 2023-01-19 09:32:00 qty: 338613 prc: 134.77 order: AAPL 2023-01-19 09:31:00 qty: 338613 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:30.018 __call__] Trade: 7023 2023-01-30T09:34 AAPL 2023-01-30 09:34:00 qty: -492461 prc: 144.64 order: AAPL 2023-01-30 09:33:00 qty: -492461 STOPPED_OUT OrderStatus.OPEN
[2023-10-18 15:32:30.019 __call__] Trade: 4374 2023-01-19T10:55 AAPL 2023-01-19 10:55:00 qty: -338613 prc: 133.92 order: AAPL 2023-01-19 10:54:00 qty: -338613 STOPPED_OUT OrderStatus.OPEN
[2023-10-18 15:32:30.021 __call__] Trade: 66

[2023-10-18 15:32:30.153 __call__] Trade: 3509 2023-01-13T16:00 AAPL 2023-01-13 16:00:00 qty: -181359 prc: 134.74 order: AAPL 2023-01-13 15:59:00 qty: -181359 EOD OrderStatus.OPEN
[2023-10-18 15:32:30.153 __call__] Trade: 3899 2023-01-17T16:00 AAPL 2023-01-17 16:00:00 qty: -266755 prc: 136 order: AAPL 2023-01-17 15:59:00 qty: -266755 EOD OrderStatus.OPEN
[2023-10-18 15:32:30.152 __call__] Trade: 4681 2023-01-20T09:32 AAPL 2023-01-20 09:32:00 qty: 235780 prc: 134.575 order: AAPL 2023-01-20 09:31:00 qty: 235780 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:30.153 __call__] Trade: 781 2023-01-05T09:32 AAPL 2023-01-05 09:32:00 qty: 197542 prc: 126.505 order: AAPL 2023-01-05 09:31:00 qty: 197542 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:30.154 __call__] Trade: 1951 2023-01-10T09:32 AAPL 2023-01-10 09:32:00 qty: 199327 prc: 130.74 order: AAPL 2023-01-10 09:31:00 qty: 199327 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:30.155 __call__] Trade: 2731 2023-01-12

[2023-10-18 15:32:30.168 __call__] Trade: 5851 2023-01-25T09:32 AAPL 2023-01-25 09:32:00 qty: 273811 prc: 139.56 order: AAPL 2023-01-25 09:31:00 qty: 273811 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:30.168 __call__] Trade: 6631 2023-01-27T09:32 AAPL 2023-01-27 09:32:00 qty: 247737 prc: 144.24 order: AAPL 2023-01-27 09:31:00 qty: 247737 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:30.168 __call__] Trade: 1966 2023-01-10T09:47 AAPL 2023-01-10 09:47:00 qty: -160033 prc: 129.93 order: AAPL 2023-01-10 09:46:00 qty: -160033 STOPPED_OUT OrderStatus.OPEN
[2023-10-18 15:32:30.169 __call__] Trade: 4291 2023-01-19T09:32 AAPL 2023-01-19 09:32:00 qty: 275826 prc: 134.77 order: AAPL 2023-01-19 09:31:00 qty: 275826 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:30.168 __call__] Trade: 1966 2023-01-10T09:47 AAPL 2023-01-10 09:47:00 qty: -160033 prc: 129.93 order: AAPL 2023-01-10 09:46:00 qty: -160033 STOPPED_OUT OrderStatus.OPEN
[2023-10-18 15:32:30.169 __call__] Trad

[2023-10-18 15:32:30.181 __call__] Trade: 4681 2023-01-20T09:32 AAPL 2023-01-20 09:32:00 qty: 263007 prc: 134.575 order: AAPL 2023-01-20 09:31:00 qty: 263007 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:30.181 __call__] Trade: 2733 2023-01-12T09:34 AAPL 2023-01-12 09:34:00 qty: -130908 prc: 132.72 order: AAPL 2023-01-12 09:33:00 qty: -130908 STOPPED_OUT OrderStatus.OPEN
[2023-10-18 15:32:30.182 __call__] Trade: 7023 2023-01-30T09:34 AAPL 2023-01-30 09:34:00 qty: -644302 prc: 144.64 order: AAPL 2023-01-30 09:33:00 qty: -644302 STOPPED_OUT OrderStatus.OPEN
[2023-10-18 15:32:30.182 __call__] Trade: 3899 2023-01-17T16:00 AAPL 2023-01-17 16:00:00 qty: -195645 prc: 136 order: AAPL 2023-01-17 15:59:00 qty: -195645 EOD OrderStatus.OPEN
[2023-10-18 15:32:30.182 __call__] Trade: 4683 2023-01-20T09:34 AAPL 2023-01-20 09:34:00 qty: -263007 prc: 134.506 order: AAPL 2023-01-20 09:33:00 qty: -263007 STOPPED_OUT OrderStatus.OPEN
[2023-10-18 15:32:30.183 __call__] Trade: 3899 2023-01-17T16:0

[2023-10-18 15:32:30.199 __call__] Trade: 5851 2023-01-25T09:32 AAPL 2023-01-25 09:32:00 qty: 315854 prc: 139.56 order: AAPL 2023-01-25 09:31:00 qty: 315854 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:30.200 __call__] Trade: 5851 2023-01-25T09:32 AAPL 2023-01-25 09:32:00 qty: 318831 prc: 139.56 order: AAPL 2023-01-25 09:31:00 qty: 318831 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:30.200 __call__] Trade: 5071 2023-01-23T09:32 AAPL 2023-01-23 09:32:00 qty: 181099 prc: 138.892 order: AAPL 2023-01-23 09:31:00 qty: 181099 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:30.200 __call__] Trade: 7021 2023-01-30T09:32 AAPL 2023-01-30 09:32:00 qty: 644302 prc: 145.345 order: AAPL 2023-01-30 09:31:00 qty: 644302 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-18 15:32:30.200 __call__] Trade: 5916 2023-01-25T10:37 AAPL 2023-01-25 10:37:00 qty: -315854 prc: 138.96 order: AAPL 2023-01-25 10:36:00 qty: -315854 STOPPED_OUT OrderStatus.OPEN
[2023-10-18 15:32:30.201 __cal

In [10]:
optimizer.plot_3d(x='stop_pct', y='ret_threshold', height=1500);

There seems to be a region around a lookback period of 35 bars and 3 standard deviations that has a positive sharpe and sortino.  The actual points you provided to the plot are shown with markers.

Let's build a contour plot to see the same data in a different view.

Lets look at the actual values of the sharpes, sortinos and drawdowns

In [4]:
df = optimizer.df_experiments(ascending = False)

In [5]:
pq.set_defaults(df_display_max_rows = 400)

In [6]:
optimizer.df_experiments(sort_column = 'maxdd', ascending = False)

,stop_pct,ret_threshold,cost,maxdd,sortino,num_trades
7,0.001,0.004,2.57819261,0.914055758,7.19979603,26
6,0.001,0.003,2.64884399,0.909936601,7.34495515,24
5,0.001,0.002,1.26943637,0.84004861,2.85771237,20
13,0.002,0.004,2.0471525,0.634423465,5.80396179,26
10,0.002,0.003,2.1165879,0.625867354,5.95500682,24
4,0.002,0.002,0.585802597,0.521284105,1.32868426,20
1,0.001,0,3.66068184,0.485136171,10.9041325,14
3,0.001,0.001,3.42381017,0.485136093,10.2490453,16
15,0.003,0.004,4.5187174,0.383053929,14.9665649,26
14,0.003,0.003,4.5632342,0.375275527,15.034623,24


Let's run the strategy at a lookback period of 35 bars with 3 standard deviations to verify the results

In [7]:
print(df.query('stop_pct == "0.05"'))
df.sort_values(by=['stop_pct', 'ret_threshold'])

Empty DataFrame
Columns: [stop_pct, ret_threshold, cost, maxdd, sortino, num_trades]
Index: []


,stop_pct,ret_threshold,cost,maxdd,sortino,num_trades
1,0.001,0,3.66068184,0.485136171,10.9041325,14
3,0.001,0.001,3.42381017,0.485136093,10.2490453,16
5,0.001,0.002,1.26943637,0.84004861,2.85771237,20
6,0.001,0.003,2.64884399,0.909936601,7.34495515,24
7,0.001,0.004,2.57819261,0.914055758,7.19979603,26
0,0.002,0,2.86901804,0.242567768,8.41999448,14
2,0.002,0.001,2.64074801,0.242568056,7.80706105,16
4,0.002,0.002,0.585802597,0.521284105,1.32868426,20
10,0.002,0.003,2.1165879,0.625867354,5.95500682,24
13,0.002,0.004,2.0471525,0.634423465,5.80396179,26


In [8]:
strategy = build_example_strategy(SimpleNamespace(lookback_period = 35, num_std = 3))
strategy.run()
strategy.evaluate_returns();

TypeError: unsupported operand type(s) for -: 'int' and 'types.SimpleNamespace'

This confirms that lookback period somewhere around 35 bars with 3 standard deviations gives us a positive sharpe and sortino and lower drawdowns.  This is a toy example, and in real life you would want to run with a lot more data, look at the stability of this region and do other out of sample testing before you can use this strategy with real money.

In [20]:
import itertools
import plotly.graph_objects as go
from plotly.subplots import make_subplots
self = optimizer
x = 'ret_threshold'
y = 'stop_pct'
z = 'all'
markers = True
filter_func = None
height = 1000
width = 0
xlim = None
ylim = None
vertical_spacing = 0.1
show = True

# Get rid of nans
experiments = [experiment for experiment in self.experiments if experiment.valid()]

xvalues = np.array([experiment.suggestion[x] for experiment in experiments])
yvalues = np.array([experiment.suggestion[y] for experiment in experiments])
zvalues = []

if z == 'all':
    zvalues.append(('cost', np.array([experiment.cost for experiment in experiments])))
    if len(experiments[0].other_costs):
        other_cost_keys = experiments[0].other_costs.keys()
        for key in other_cost_keys:
            zvalues.append((key, np.array([experiment.other_costs[key] for experiment in experiments])))
elif z == 'cost':
    zvalues.append(('cost', np.array([experiment.cost for experiment in experiments])))
else:
    zvalues.append((z, np.array([experiment.other_costs[z] for experiment in experiments])))

cols: dict[str, np.ndarray] = dict(x=xvalues, y=yvalues)
for metric, _z in zvalues:
    cols[metric] = _z
_df = pd.DataFrame(cols).sort_values(by=['x', 'y'])
_x = np.unique(_df.x.values)
_y = np.unique(_df.y.values)
df = _df.set_index(['x', 'y']).reindex(itertools.product(_x, _y)).reset_index()

metrics = np.unique([metric[0] for metric in zvalues])
_z = np.full((len(metrics), len(_x), len(_y)), np.nan)

for i, metric in enumerate(metrics):
    _z[i, :, :] = df[metric].values.reshape((len(_x), len(_y)))

fig = make_subplots(rows=len(metrics), cols=1, subplot_titles=metrics, shared_xaxes=True, vertical_spacing=vertical_spacing)
fig.update_layout(height=height)

num_metrics = len(metrics)
colorbar_height = 1 / (num_metrics + 1)

for i, metric in enumerate(metrics):
    print(i)
    zmatrix = _z[i]
    row = i + 1
    min_z = np.nanmin(zmatrix)
    max_z = np.nanmax(zmatrix)

    zero: float = np.nan
    if np.sign(min_z) != np.sign(max_z):
        print(max_z, min_z)
        zero = (0 - min_z) / (max_z - min_z)
        colorscale = [(0, 'rgba(237, 100, 90, 0.85)'),
                      (zero, 'white'),
                      (1, 'rgba(17, 165, 21, 0.85)')]
    elif min_z <= 0:  # both min and max are negative
        colorscale = [(0, 'rgba(237, 100, 90, 0.85)'),
                      (1, 'white')]
    else:  # both min and max are positive
        colorscale = [(0, 'white'),
                      (1, 'rgba(17, 165, 21, 0.85)')]

    colorbar_y = 1 - (i + 1) * colorbar_height
    print('adding trace')
    trace = go.Contour(x=_x, 
                       y=_y, 
                       z=zmatrix, 
                       name=metric, 
                       colorscale=colorscale, 
                       colorbar=dict(len=colorbar_height, y=colorbar_y),
                       connectgaps=True,
                       contours=dict(showlabels=True, labelfont=dict(color='white')));
    fig.update_xaxes(title_text=x, row=row, col=1);
    fig.update_yaxes(title_text=y, row=row, col=1);

    if markers:
        scatter = go.Scatter(x=_x, y=_y, marker_color='red', marker_size=10, marker_symbol='triangle-down', mode='markers');
        fig.add_trace(scatter, row=row, col=1);
    fig.add_trace(trace, row=row, col=1);

    fig.update_layout(showlegend=False);
    fig.show();



0
adding trace


1
adding trace


2
adding trace


3
adding trace


In [22]:
_x = np.unique(df.x.values)
_y = np.unique(df.y.values)
fig = go.Figure()
trace = go.Contour(x=_x, 
                   y=_y, 
                   z=zmatrix);

scatter = go.Scatter(x=_x, y=_y, marker_color='red', marker_size=10, marker_symbol='triangle-down', mode='markers');
fig.add_trace(trace);
fig.add_trace(scatter);
fig.show();
